In [20]:
from tqdm.notebook import trange, tqdm
from tba_types import Award, Event, EliminationAlliance
from collections import defaultdict
from tba_session import session, ROOT_URL
import json

In [23]:
EXTRA_ALLIANCES: dict[str, list[list[str]]] = {}

with open("extra_alliances.json", "r") as f:
    to_process = json.load(f)
    EXTRA_ALLIANCES = {
        key: [
            ["frc" + team.strip() for team in line.split("-")]
            for line in string.splitlines()
        ]
        for key, string in to_process.items()
    }

print(EXTRA_ALLIANCES)

{'2011co': [['frc1986', 'frc624', 'frc2972'], ['frc662', 'frc2996', 'frc1977'], ['frc1619', 'frc2036', 'frc3648'], ['frc1515', 'frc1158', 'frc3729'], ['frc1332', 'frc1157', 'frc2945'], ['frc159', 'frc3403', 'frc2083'], ['frc1348', 'frc2352', 'frc1339'], ['frc443', 'frc1410', 'frc1245']], '2011da': [['frc148', 'frc2848', 'frc3037'], ['frc1296', 'frc704', 'frc3795'], ['frc647', 'frc3802', 'frc3005'], ['frc3507', 'frc1817', 'frc3384'], ['frc3310', 'frc418', 'frc3043'], ['frc3666', 'frc2949', 'frc3350'], ['frc3369', 'frc2995', 'frc1745'], ['frc3370', 'frc2283', 'frc2864']]}


In [58]:
TEAM_EVENT_PICK: dict[str, dict[str, int]] = defaultdict(dict)
EVENT_KEY_MAP: dict[str, str] = {}
EVENT_MAP: dict[str, Event] = {}

for year in trange(2010, 2026):
    if year == 2021:
        continue
    events: list[Event] = session.get(f"{ROOT_URL}/events/{year}").json()
    for event in tqdm(sorted(events, key=lambda e: e["start_date"]), desc=f"Processing {year} events", leave=False):
        EVENT_KEY_MAP[event["key"]] = event["parent_event_key"] or event["key"]
        EVENT_MAP[event["key"]] = event
        if (
            event["event_type"] >= 6
            or "cancelled" in event["name"].lower()
            or event["event_code"].startswith("award")
            or event["event_code"].startswith("cmp")
        ):
            continue
        teams: list[str] = session.get(
            f"{ROOT_URL}/event/{event['key']}/teams/keys"
        ).json()
        for team in teams:
            if team.endswith("*"):
                team = team[:-1]
            if EVENT_KEY_MAP[event["key"]] not in TEAM_EVENT_PICK[team]:
                TEAM_EVENT_PICK[team][EVENT_KEY_MAP[event["key"]]] = -1
        alliances: list[EliminationAlliance] = session.get(
            f"{ROOT_URL}/event/{event['key']}/alliances"
        ).json()
        if not alliances:
            if event["key"] in EXTRA_ALLIANCES:
                for alliance in EXTRA_ALLIANCES[event["key"]]:
                    for i, team in enumerate(alliance):
                        TEAM_EVENT_PICK[team][EVENT_KEY_MAP[event["key"]]] = i
                continue
            print(f"No alliances for {event['key']} ({event["name"]}), skipping")
            continue
        for alliance in alliances:
            if alliance["picks"]:
                for i, team in enumerate(alliance["picks"]):
                    if team.endswith("*"):
                        team = team[:-1]
                    TEAM_EVENT_PICK[team][EVENT_KEY_MAP[event["key"]]] = i

  0%|          | 0/16 [00:00<?, ?it/s]

Processing 2010 events:   0%|          | 0/58 [00:00<?, ?it/s]

No alliances for 2010az (Arizona Regional), skipping
No alliances for 2010is (Israel Regional), skipping
No alliances for 2010in (Boilermaker Regional), skipping
No alliances for 2010ut (Utah Regional sponsored by NASA & Platt), skipping
No alliances for 2010co (Colorado Regional), skipping
No alliances for 2010on (Greater Toronto Regional), skipping


Processing 2011 events:   0%|          | 0/68 [00:00<?, ?it/s]

No alliances for 2011az (Arizona Regional), skipping
No alliances for 2011ok (Oklahoma Regional), skipping
No alliances for 2011sac (Sacramento Regional), skipping
No alliances for 2011tx (Lone Star Regional), skipping
No alliances for 2011ut (Utah Regional co-sponsored by NASA and Platt), skipping


...

No alliances for 2022zhha (Nanjing Regional), skipping


Processing 2023 events:   0%|          | 0/309 [00:00<?, ?it/s]

No alliances for 2023tuis3 (Izmir Regional), skipping


Processing 2024 events:   0%|          | 0/323 [00:00<?, ?it/s]

Processing 2025 events:   0%|          | 0/222 [00:00<?, ?it/s]

In [57]:
TEAM_EVENT_PICK["frc125"]["2025necmp"]

-1

In [59]:
def longest_streak(pick_range: range) -> dict[str, tuple[int, dict[str, int]]]:
    longest_streak: dict[str, tuple[int, dict[str, int]]] = {}
    for team_key, events in TEAM_EVENT_PICK.items():
        longest_streak[team_key] = (0, {})
        items = list(events.items())
        start = -1
        for end in range(len(items)):
            if items[end][1] not in pick_range:
                start = -1
                continue
            elif start == -1:
                start = end
            else:
                while items[start][1] not in pick_range:
                    start += 1
                    if start >= end:
                        break
            if end - start + 1 > longest_streak[team_key][0]:
                longest_streak[team_key] = (
                    end - start + 1,
                    dict(items[start : end + 1]),
                )
    return longest_streak

In [63]:
print("| Team | Length | Active | First Event | Last Event |")
print("| ---- | ------ | ------ | ----------- | ----------- |")

min_length = 30
for team_key, vals in sorted(
    longest_streak(range(0, 2)).items(),
    key=lambda x: (x[1][0], 20000 - int(x[0][3:])),
    reverse=True,
):
    length, picks = vals
    event_list = list(picks.keys())
    if length < min_length:
        break
    active = event_list[-1] == list(TEAM_EVENT_PICK[team_key].keys())[
        -1
    ] and event_list[-1].startswith("2025")
    print(f"| [{team_key[3:]}](https://www.thebluealliance.com/team/{team_key[3:]}) | {length} | {"Yes" if active else "No"} |"
            f"[{EVENT_MAP[event_list[0]]["year"]} {EVENT_MAP[event_list[0]]["name"]}](https://www.thebluealliance.com/event/{event_list[0]}) |"
            f"[{EVENT_MAP[event_list[-1]]["year"]} {EVENT_MAP[event_list[-1]]["name"]}](https://www.thebluealliance.com/event/{event_list[-1]}) |")

| Team | Length | Active | First Event | Last Event |
| ---- | ------ | ------ | ----------- | ----------- |
| [67](https://www.thebluealliance.com/team/67) | 61 | Yes |[2010 Kettering University FIRST Robotics District Competition](https://www.thebluealliance.com/event/2010gg) |[2025 Einstein Field](https://www.thebluealliance.com/event/2025cmptx) |
| [118](https://www.thebluealliance.com/team/118) | 61 | Yes |[2011 Midwest Regional](https://www.thebluealliance.com/event/2011il) |[2025 Einstein Field](https://www.thebluealliance.com/event/2025cmptx) |
| [33](https://www.thebluealliance.com/team/33) | 53 | No |[2010 Kettering University FIRST Robotics District Competition](https://www.thebluealliance.com/event/2010gg) |[2023 FIRST in Michigan State Championship presented by DTE Foundation](https://www.thebluealliance.com/event/2023micmp) |
| [2056](https://www.thebluealliance.com/team/2056) | 52 | Yes |[2010 Einstein Field](https://www.thebluealliance.com/event/2010cmp) |[2025 Einstein

In [ ]:
if "get_ipython" not in globals():
    import json
    import pathlib

    output_folder = pathlib.Path(__file__).parent / "build"

    config = {
        "columns": [
            {
                "name": "team",
                "type": "team",
                "displayName": "Team",
            },
            {
                "name": "length",
                "type": "integer",
                "displayName": "Streak Length",
            },
            {
                "name": "event_start",
                "type": "event",
                "displayName": "Start Event",
            },
            {
                "name": "event_end",
                "type": "event",
                "displayName": "End Event",
            },
            {
                "name": "active",
                "type": "boolean",
                "displayName": "Active",
            },
        ],
        "defaultSort": [{"column": "length", "direction": -1}],
        "primaryKey": "team",
    }

    with (output_folder / "alliance-captain-streaks.config.json").open("w") as f:
        json.dump({**config, "title": "Alliance Captain Streaks"}, f, indent=None)
    with (output_folder / "alliance-captain-streaks.json").open("w") as f:
        json.dump(
            [
                {
                    "team": team_key,
                    "length": vals[0],
                    "event_start": list(vals[1].keys())[0],
                    "event_end": list(vals[1].keys())[-1],
                    "active": list(vals[1].keys())[-1]
                    == list(TEAM_EVENT_PICK[team_key].keys())[-1]
                    and list(vals[1].keys())[-1].startswith("2025"),
                }
                for team_key, vals in longest_streak(range(1)).items()
                if list(vals[1].keys())
            ],
            f,
            indent=None,
        )
    with (output_folder / "captain-first-pick.config.json").open("w") as f:
        json.dump(
            {
                **config,
                "title": "Captain/1st Pick Streaks",
            },
            f,
            indent=None,
        )
    with (output_folder / "captain-first-pick.json").open("w") as f:
        json.dump(
            [
                {
                    "team": team_key,
                    "length": vals[0],
                    "event_start": list(vals[1].keys())[0],
                    "event_end": list(vals[1].keys())[-1],
                    "active": list(vals[1].keys())[-1]
                    == list(TEAM_EVENT_PICK[team_key].keys())[-1]
                    and list(vals[1].keys())[-1].startswith("2025"),
                }
                for team_key, vals in longest_streak(range(2)).items()
                if list(vals[1].keys())
            ],
            f,
            indent=None,
        )